<a href="https://colab.research.google.com/github/dthomp12/MLBModel/blob/master/MLBModelPredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
%%capture
!apt-get update # to update ubuntu to correctly run apt install
!pip3 install selenium
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [137]:
%%capture
link = "https://www.covers.com/sport/baseball/mlb/odds?stateProvince=Pennsylvania&countryCode=US"
from selenium import webdriver
import time
import pandas as pd
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
wd.get(link)
time.sleep(.5)
html = wd.page_source
wd.close()

dfs = pd.read_html(html)

In [151]:
teams = dfs[1].iloc[:,1]
MLs = dfs[3].iloc[:,:3]
Spreads = dfs[7].iloc[:,:3]
vegasOdds = pd.concat([teams, MLs, Spreads], axis=1)

In [111]:
# Methods to get the games and pitchers for a given day

team_fixes = {"WAS":"WSN", "SF":"SFG", "TB":"TBR", "SD":"SDP", "KC":"KCR"}
team_unfixes = {v : k for k, v in team_fixes.items()}

def get_Team_data(row):
  if len(row) == 0:
    return "ARI", "TBD", "None"
  hand = 1 if row.pop(-1) == "RHP" else -1
  team = row.pop(-1)
  SP = " ".join(row)
  return team, SP, hand

def get_Info(gameTable):
  awayRow = gameTable.iloc[0]["players"].split()[4:]
  homeRow = gameTable.iloc[1]["players"].split()[4:]
  awayTeam, awaySP, awayHand = get_Team_data(awayRow)
  homeTeam, homeSP, homeHand = get_Team_data(homeRow)
  if awayTeam in team_fixes.keys():
    awayTeam = team_fixes[awayTeam]
  if homeTeam in team_fixes.keys():
    homeTeam = team_fixes[homeTeam]
  return [awayTeam, awaySP, awayHand, homeTeam, homeSP, homeHand]


def get_Games(date):
  dateString = date.replace("-", "")
  link = f"https://www.cbssports.com/fantasy/baseball/probable-pitchers/{dateString}"
  games = []

  dfs = pd.read_html(link)
  for game in dfs:
    games.append(get_Info(game))

  return games

In [112]:
# Methods to get the model inputs for a game
import pandas as pd
import time
import math
from datetime import date, timedelta
from selenium import webdriver

team_num = {"ARI": "15", "ATL": "16", "BAL": "2", "BOS": "3", "CHC": "17",
            "CHW": "4", "CIN": "18", "CLE": "5", "COL": "19", "DET": "6", "HOU": "21"
    , "KCR": "7", "LAA": "1", "LAD": "22", "MIA": "20", "MIL": "23", "MIN": "8"
    , "NYM": "25", "NYY": "9", "OAK": "10", "PHI": "26", "PIT": "27", "SDP": "29"
    , "SEA": "11", "SFG": "30", "STL": "28", "TBR": "12", "TEX": "13", "TOR": "14", "WSN": "24"}

def offense_link(startDate, endDate, rightHandPitcher):
    rOrL = "2" if rightHandPitcher else "1"
    return 'https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=' + rOrL + '&splitArrPitch=&position=B&autoPt=false&splitTeams=false&statType=team&statgroup=2&startDate=' + startDate + "&endDate=" + endDate + "&players=&filter=&groupBy=season&wxTemperature=&wxPressure=&wxAirDensity=&wxElevation=&wxWindSpeed=&sort=23,1"

def get_dynamic_df(link):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  wd.get(link)
  time.sleep(.5)
  html = wd.page_source
  wd.close()

  return pd.read_html(html)[-1]

def getdfVright(date):
  link = offense_link("2022-03-01", date, True)
  return get_dynamic_df(link)
  

def getdfVleft(date):
  link = offense_link("2022-03-01", date, False)
  return get_dynamic_df(link)

def getdfRP(year):
  link = f"https://www.fangraphs.com/leaders.aspx?pos=all&stats=rel&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0,ts&rost=0&age=0&filter=&players=0&startdate=&enddate="
  return pd.read_html(link)[16]

def getRPStats(team, df):
  for index, row in df.iterrows():
    if row.iloc[1] == team:
      return row.iloc[19]
  return None

def getdfBSR(year):
  link = f"https://www.fangraphs.com/leaders.aspx?pos=all&stats=bat&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team=0,ts&rost=0&age=0&filter=&players=0&startdate=&enddate="
  return pd.read_html(link)[16]

def getBSR(team, df):
  for index, row in df.iterrows():
    if row.iloc[1] == team:
      return row.iloc[18]
  return None

def getHittingStats(dfR, dfL, team):
  wrcR, wrcL = 0, 0
  for index, row in dfR.iterrows():
    if row["Tm"] == team:
      wrcR = row["wRC+"]
      break
  for index, row in dfL.iterrows():
    if row["Tm"] == team:
      wrcL = row["wRC+"]
      break
  return wrcR, wrcL

def getFieldingStats(team, Year):
  num = team_num[team]
  link = f"https://www.fangraphs.com/leaders.aspx?pos=all&stats=fld&lg=all&qual=0&type=1&season={Year}&month=0&season1={Year}&ind=0&team={num},ts&rost=0&age=0&filter=&players=0&startdate=&enddate="
  df = pd.read_html(link)[16]
  FRM = df.iloc[0][18]
  OAA = df.iloc[0][19]
  return FRM, OAA

def getSPStats(team, sp, year):
  link = f"https://www.fangraphs.com/leaders.aspx?pos=all&stats=sta&lg=all&qual=0&type=8&season={year}&month=0&season1={year}&ind=0&team={team_num[team]}&rost=0&age=0&filter=&players=0&startdate={year}-01-01&enddate={year}-12-31"
  df = pd.read_html(link)[16]
  xFIP, G, IP = None, None, None
  for index, row in df.iterrows():
    if row.iloc[1] == sp:
      xFIP = row.iloc[19]
      G = row.iloc[5]
      IP = row.iloc[7]
      break
  if xFIP is None:
    sp = input(f"{sp} not found. Enter name if error or X if first start\n").strip()
    if sp == "X":
      raise Exception
    else:
      return getSPStats(team, sp, year)
  IP = float(IP)
  IP = math.floor(IP) + (IP % 1) * 1 / 3
  return float(xFIP), round(IP / float(G), 3)

In [113]:
%%capture

from tensorflow.keras.models import load_model
spreadModel1 = load_model("drive/MyDrive/MLBModels/spreadModel1")
spreadModel2 = load_model("drive/MyDrive/MLBModels/spreadModel2")
MLModel = load_model("drive/MyDrive/MLBModels/MLModel1")

date = input("What is the date? (YYYY-MM-DD)    ").strip()
year = "2022"

games = get_Games(date)
dfVright = getdfVright(date)
dfVleft = getdfVleft(date)
dfBSR = getdfBSR(year)
dfRP = getdfRP(year)

What is the date? (YYYY-MM-DD)    2022-05-29


In [114]:
def hitting(wrcL, wrcR, oppHand, oppSPFIP, oppRPFIP, oppSPIPG):
  oppSPFIP = 4.26 / oppSPFIP * 100 #4.26 is average FIP
  oppRPFIP = 4.316 / oppRPFIP * 100 #4.316 if average RP FIP
  if oppHand == 1: #rightie SP
    spHitting = wrcR - oppSPFIP
  elif oppHand == -1: #leftie SP
    spHitting = wrcL - oppSPFIP
  else: #switch pitcher
    spHitting = np.min(wrcL, wrcR) - oppSPFIP
  rpHitting = .66 * wrcR + .34 * wrcL - oppRPFIP # assume 66% of relievers are righties
  hitting = (oppSPIPG * spHitting + (9-oppSPIPG) * rpHitting) / 9
  return hitting

def get_Score(OAA, FRM, BSR, wrcL, wrcR, oppHand, oppSPFIP, oppRPFIP, oppSPIPG):
  oaa_weight, frm_weight, bsr_weight = .1187, .436, .93
  return oaa_weight*OAA + frm_weight*FRM + bsr_weight*BSR + hitting(wrcL, wrcR, oppHand, oppSPFIP, oppRPFIP, oppSPIPG) 

def predict_game(gameLine):
  hWRCL,hWRCR,hFRM,hOAA,hBsR,hHand,hSpxFIP,hRpxFIP,hSpIperG,aWRCL,aWRCR,aFRM,aOAA,aBsR,aHand,aSpxFIP,aRpxFIP,aSpIperG = gameLine
  homeScore = get_Score(hOAA, hFRM, hBsR, hWRCL, hWRCR, aHand, aSpxFIP, aRpxFIP, aSpIperG)
  awayScore = get_Score(aOAA, aFRM, aBsR, aWRCL, aWRCR, hHand, hSpxFIP, hRpxFIP, hSpIperG)
  score_diff = homeScore - awayScore
  spread_pred = .1769 * (np.sign(score_diff) * np.abs(score_diff)**(.4918)) - 0.196
  homeWin_prob = .0171559 * (np.sign(score_diff) * np.abs(score_diff)**.5) + .505246
  return homeWin_prob, spread_pred

In [157]:
vegasOdds

,Unnamed: 1,Unnamed: 0,Unnamed: 1,Unnamed: 2,0,1,2
0,MIL (Burnes) Corbin Burnes STL (Mikolas) ...,-154 +130,-154 +135,-145 +125,-1.5 +115 +1.5 -138,-1.5 +115 +1.5 -135,-1.5 +120 +1.5 -140
1,TEX (Dunning) Dane Dunning OAK (Kaprielian...,-120 +102,-125 +105,-125 +105,-1.5 +146 +1.5 -178,-1.5 +140 +1.5 -161,-1.5 +140 +1.5 -160
2,TOR (Berrios) Jose Berrios LAA (Sandoval) ...,+132 -156,+135 -154,+135 -155,+1.5 -170 -1.5 +140,+1.5 -169 -1.5 +145,+1.5 -170 -1.5 +150
3,LAD (Anderson) Tyler Anderson ARI (Davies)...,-215 +180,-213 +185,-200 +170,-1.5 -128 +1.5 +106,-1.5 -135 +1.5 +115,-1.5 -130 +1.5 +110
4,PIT (Contreras) Roansy Contreras SD (Gore)...,+150 -178,+155 -175,+155 -180,+1.5 -150 -1.5 +125,+1.5 -141 -1.5 +120,+1.5 -135 -1.5 +115
5,HOU (Garcia) Luis Garcia SEA (Gonzales) M...,-154 +130,-145 +125,-140 +120,-1.5 +104 +1.5 -125,-1.5 +110 +1.5 -130,-1.5 +120 +1.5 -140
6,PHI (Wheeler) Zack Wheeler NYM (Bassitt) ...,+100 -118,-105 -115,-105 -115,-1.5 +176 +1.5 -215,+1.5 -208 -1.5 +175,-1.5 +165 +1.5 -195
7,KC (Heasley) Jonathan Heasley CLE (Plesac)...,NaN,NaN,+145 -165,NaN,NaN,+1.5 -140 -1.5 +120
8,BAL (Wells) Tyler Wells BOS (Hill) Rich Hill,NaN,NaN,+120 -140,NaN,NaN,+1.5 -160 -1.5 +140
9,SF (Cobb) Alex Cobb 4 CIN (Mahle) Tyler ...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import numpy as np
from scipy.integrate import quad as integral

vegasOdds2 = vegasOdds.copy()

def prob_to_odds(prob, edge):
  prob -= edge
  if prob >= .5:
    return np.round(-(prob / (1-prob)) * 100, 0)
  else:
    return np.round((1-prob)/(prob) * 100, 0)

def odds_to_prob(odds):
  if odds < 0:
    return -odds / (-odds + 100)
  else:
    return 100 / (odds + 100)

def laplace_dist(x, mean, b):
    output = 0
    for pred in mean:
      output += (1 / (2*b) * np.exp(-np.abs(x-pred)/b)) / len(mean)
    return output

def prob_plus(preds):
  std = 5
  homePlus = integral(laplace_dist, -1.5, np.inf, args=(preds, std))[0]
  awayMinus = 1 - homePlus
  return homePlus, awayMinus

def prob_minus(preds):
  std = 5
  homeMinus = integral(laplace_dist, 1.5, np.inf, args=(preds, std))[0]
  awayPlus = 1 - homeMinus
  return homeMinus, awayPlus

def normalize(probs):
  awayP, homeP = probs
  factor = awayP + homeP
  return awayP / factor, homeP / factor

def get_Vegas_ML(row):
  awayOdds = []
  homeOdds = []
  odds = row.iloc[1:4]
  for site in odds:
    if type(site) != type("string"):
      homeOdds.append(-1000)
      awayOdds.append(-1000)
    else:
      awayOdd, homeOdd = site.split()
      awayOdds.append(int(awayOdd))
      homeOdds.append(int(homeOdd))
  maxAwayOdd = max(awayOdds)
  maxHomeOdd = max(homeOdds)
  return maxAwayOdd, maxHomeOdd, awayOdds.index(maxAwayOdd), homeOdds.index(maxHomeOdd)

def get_Vegas_spread(row):
  awayPlusOdds = []
  awayMinusOdds = []
  homePlusOdds = []
  homeMinusOdds = []
  odds = row.iloc[4:]
  for site in odds:
    if type(site) != type("string"):
      homePlusOdds.append(-1000)
      homeMinusOdds.append(-1000)
      awayPlusOdds.append(-1000)
      awayMinusOdds.append(-1000)
    else:
      pOrMA, awayOdd, pOrMH, homeOdd = site.split()
      if pOrMA == "-1.5":
        homePlusOdds.append(int(homeOdd))
        homeMinusOdds.append(-1000)
        awayPlusOdds.append(-1000)
        awayMinusOdds.append(int(awayOdd))
      else:
        homePlusOdds.append(-1000)
        homeMinusOdds.append(int(homeOdd))
        awayPlusOdds.append(int(awayOdd))
        awayMinusOdds.append(-1000)
      
  maxAwayPOdd = max(awayPlusOdds)
  maxHomePOdd = max(homePlusOdds)
  maxAwayMOdd = max(awayMinusOdds)
  maxHomeMOdd = max(homeMinusOdds)
  return maxAwayPOdd, maxAwayMOdd, maxHomePOdd, maxHomeMOdd, awayPlusOdds.index(maxAwayPOdd), awayMinusOdds.index(maxAwayMOdd), homePlusOdds.index(maxHomePOdd),homeMinusOdds.index(maxHomeMOdd)

def get_Vegas_Odds(awayTeam, homeTeam):
  if awayTeam in team_unfixes.keys():
    awayTeam = team_unfixes[awayTeam]
  if homeTeam in team_unfixes.keys():
    homeTeam = team_unfixes[homeTeam]
  for index, row in vegasOdds2.iterrows():
    if awayTeam in row.iloc[0] and homeTeam in row.iloc[0]:
      lines = get_Vegas_ML(row)
      awayML, homeML, MLsiteAway, MLsiteHome = lines
      lines = get_Vegas_spread(row)
      awayPSpread, awayMSpread, homePSpread, homeMSpread, awayPSpreadsite, awayMSpreadsite, homePSpreadsite, homeMSpreadsite = lines
      vegasOdds2.drop([index], axis=0, inplace=True)
      return awayML, homeML, MLsiteAway, MLsiteHome, awayPSpread, awayMSpread, homePSpread, homeMSpread, awayPSpreadsite, awayMSpreadsite, homePSpreadsite, homeMSpreadsite
        #best_Spread, SpreadsiteAway, SpreadsiteHome = get_Vegas_spread(row)
  return [None for x in range(12)]

gameInputs = []

NN_MLbets = []
Excel_MLbets = []
NN_Spreadbets = []
Excel_Spreadbets = []

for game in games:
  
  awayTeam, homeTeam = game[0], game[3]
  awaySP, homeSP = game[1], game[4]
  aHand, hHand = game[2], game[5]

  awayML, homeML, awayMLsite, homeMLsite, awayPSpread, awayMSpread, homePSpread, homeMSpread, awayPSpreadsite, awayMSpreadsite, homePSpreadsite, homeMSpreadsite = get_Vegas_Odds(awayTeam, homeTeam)

  if awayML is None:
    continue

  awayVegasProb, homeVegasProb = odds_to_prob(awayML), odds_to_prob(homeML)
  awayPSpreadProbV, awayMSpreadProbV, homePSpreadProbV, homeMSpreadProbV = odds_to_prob(awayPSpread), odds_to_prob(awayMSpread), odds_to_prob(homePSpread), odds_to_prob(homeMSpread)

  try:
    hSpxFIP, hSpIperG = getSPStats(homeTeam, homeSP, year)
  except Exception:
    print(f"{awayTeam} vs {homeTeam}:\n{homeSP} not found\n")
    continue
  try:
    aSpxFIP, aSpIperG = getSPStats(awayTeam, awaySP, year)
  except Exception:
    print(f"{awayTeam} vs {homeTeam}:\n{awaySP} not found\n")
    continue

  hWRCL, hWRCR = getHittingStats(dfVright, dfVleft, homeTeam)
  aWRCL, aWRCR = getHittingStats(dfVright, dfVleft, awayTeam)

  hFRM, hOAA = getFieldingStats(homeTeam, year)
  aFRM, aOAA = getFieldingStats(awayTeam, year)

  hBsR = getBSR(homeTeam, dfBSR)
  aBsR = getBSR(awayTeam, dfBSR)

  hRpxFIP = getRPStats(homeTeam, dfRP)
  aRpxFIP = getRPStats(awayTeam, dfRP)

  gameList = [hWRCL,hWRCR,hFRM,hOAA,hBsR,hHand,hSpxFIP,hRpxFIP,hSpIperG,aWRCL,aWRCR,aFRM,aOAA,aBsR,aHand,aSpxFIP,aRpxFIP,aSpIperG]

  for i in range(len(gameList)):
    gameList[i] = float(gameList[i])

  gameList = np.array(gameList)


  # NN ML predictions
  pred = MLModel.predict(np.array([gameList]))

  awayProb, homeProb = normalize(pred[0])

  # Find positive edges
  if (awayProb - awayVegasProb > 0):
    NN_MLbets.append(f"{date}\t{awayTeam}\t{'+' if awayML > 0 else ''}{awayML}\t{(awayProb - awayVegasProb) * 100:.2f}%\t{awayMLsite}")
  if (homeProb - homeVegasProb > 0):
    NN_MLbets.append(f"{date}\t{homeTeam}\t{'+' if homeML > 0 else ''}{homeML}\t{(homeProb - homeVegasProb) * 100:.2f}%\t{homeMLsite}")

  # NN Spread predictions
  spreadPreds = [spreadModel1.predict(np.array([gameList]))[0][0], spreadModel2.predict(np.array([gameList]))[0][0]]

  spreadPred = (spreadPreds[0] + spreadPreds[1])/2

  homePlus, awayMinus = prob_plus(spreadPreds)
  homeMinus, awayPlus = prob_minus(spreadPreds)

  # Find positive edges
  if (awayPlus - awayPSpreadProbV > 0):
    NN_Spreadbets.append(f"{date}\t{awayTeam}+1.5\t{'+' if awayPSpread > 0 else ''}{awayPSpread}\t{(awayPlus - awayPSpreadProbV) * 100:.2f}%\t{awayPSpreadsite}")
  if (awayMinus - awayMSpreadProbV > 0):
    NN_Spreadbets.append(f"{date}\t{awayTeam}-1.5\t{'+' if awayMSpread > 0 else ''}{awayMSpread}\t{(awayMinus - awayMSpreadProbV) * 100:.2f}%\t{awayMSpreadsite}")
  if (homePlus - homePSpreadProbV > 0):
    NN_Spreadbets.append(f"{date}\t{homeTeam}+1.5\t{'+' if homePSpread > 0 else ''}{homePSpread}\t{(homePlus - homePSpreadProbV) * 100:.2f}%\t{homePSpreadsite}")
  if (homeMinus - homeMSpreadProbV > 0):
    NN_Spreadbets.append(f"{date}\t{homeTeam}-1.5\t{'+' if homeMSpread > 0 else ''}{homeMSpread}\t{(homeMinus - homeMSpreadProbV) * 100:.2f}%\t{homeMSpreadsite}")

  # Excel ML predictions
  homeProb, spreadPreds = predict_game(gameList)
  awayProb = 1 - homeProb

  # Find positive edges
  if (awayProb - awayVegasProb > 0):
    Excel_MLbets.append(f"{date}\t{awayTeam}\t{'+' if awayML > 0 else ''}{awayML}\t{(awayProb - awayVegasProb) * 100:.2f}%\t{awayMLsite}")
  if (homeProb - homeVegasProb > 0):
    Excel_MLbets.append(f"{date}\t{homeTeam}\t{'+' if homeML > 0 else ''}{homeML}\t{(homeProb - homeVegasProb) * 100:.2f}%\t{homeMLsite}")

  # Excel Spread predictions
  homePlus, awayMinus = prob_plus([spreadPreds])
  homeMinus, awayPlus = prob_minus([spreadPreds])

  # Find positive edges
  if (awayPlus - awayPSpreadProbV > 0):
    Excel_Spreadbets.append(f"{date}\t{awayTeam}+1.5\t{'+' if awayPSpread > 0 else ''}{awayPSpread}\t{(awayPlus - awayPSpreadProbV) * 100:.2f}%\t{awayPSpreadsite}")
  if (awayMinus - awayMSpreadProbV > 0):
    Excel_Spreadbets.append(f"{date}\t{awayTeam}-1.5\t{'+' if awayMSpread > 0 else ''}{awayMSpread}\t{(awayMinus - awayMSpreadProbV) * 100:.2f}%\t{awayMSpreadsite}")
  if (homePlus - homePSpreadProbV > 0):
    Excel_Spreadbets.append(f"{date}\t{homeTeam}+1.5\t{'+' if homePSpread > 0 else ''}{homePSpread}\t{(homePlus - homePSpreadProbV) * 100:.2f}%\t{homePSpreadsite}")
  if (homeMinus - homeMSpreadProbV > 0):
    Excel_Spreadbets.append(f"{date}\t{homeTeam}-1.5\t{'+' if homeMSpread > 0 else ''}{homeMSpread}\t{(homeMinus - homeMSpreadProbV) * 100:.2f}%\t{homeMSpreadsite}")

print("Neural Net Moneyline bets:\n")
for bet in NN_MLbets:
  print(bet)

print("\nNeural Net Spread bets:\n")
for bet in NN_Spreadbets:
  print(bet)

print("\n**********************************\n")

print("Excel Moneyline bets:\n")
for bet in Excel_MLbets:
  print(bet)

print("\nExcel Spread bets:\n")
for bet in Excel_Spreadbets:
  print(bet)

In [191]:
from google.colab import auth
auth.authenticate_user()

import gspread
import Spreadsheets.
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)


In [196]:
wb = gc.open("MLB tracking Auto").worksheet("ML").append_rows([bet.split() for bet in NN_MLbets], table_range="B2:F", value_input_option="USER_ENTERED")
wb = gc.open("MLB tracking Auto").worksheet("Spread").append_rows([bet.split() for bet in NN_Spreadbets], table_range="B2:F", value_input_option="USER_ENTERED")
wb = gc.open("MLB tracking Auto").worksheet("Excel ML").append_rows([bet.split() for bet in Excel_MLbets], table_range="B2:F", value_input_option="USER_ENTERED")
wb = gc.open("MLB tracking Auto").worksheet("Excel Spread").append_rows([bet.split() for bet in Excel_Spreadbets], table_range="B2:F", value_input_option="USER_ENTERED")